In [65]:
import pandas as pd
import re
import numpy as np
possible_formats = []
def find_possible_date_formats(date_str):
    # List month name
    months = r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|January|February|March|April|May|June|July|August|September|October|November|December)'

    # Pattern to determine date format in string
    patterns = {
        'YYYY-MM-DD': r'^([2-9][0-9]{3})-(1[0-2]|0?[1-9])-(3[01]|[12][0-9]|0?[1-9])$',
        'DD-MM-YYYY': r'^(3[01]|[12][0-9]|0?[1-9])-(1[0-2]|0?[1-9])-([2-9][0-9]{3})$',
        'MM-DD-YYYY': r'^(1[0-2]|0?[1-9])-(3[01]|[12][0-9]|0?[1-9])-([2-9][0-9]{3})$',
        'YYYY/MM/DD': r'^([2-9][0-9]{3})/(1[0-2]|0?[1-9])/(3[01]|[12][0-9]|0?[1-9])$',
        'DD/MM/YYYY': r'^(3[01]|[12][0-9]|0?[1-9])/(1[0-2]|0?[1-9])/([2-9][0-9]{3})$',
        'MM/DD/YYYY': r'^(1[0-2]|0?[1-9])/(3[01]|[12][0-9]|0?[1-9])/([2-9][0-9]{3})$',
        'YYYY.MM.DD': r'^([2-9][0-9]{3})\.(1[0-2]|0?[1-9])\.(3[01]|[12][0-9]|0?[1-9])$',
        'DD.MM.YYYY': r'^(3[01]|[12][0-9]|0?[1-9])\.(1[0-2]|0?[1-9])\.([2-9][0-9]{3})$',
        'MM.DD.YYYY': r'^(1[0-2]|0?[1-9])\.(3[01]|[12][0-9]|0?[1-9])\.([2-9][0-9]{3})$',
        'DD Month YYYY': rf'^(3[01]|[12][0-9]|0?[1-9]) {months} ([2-9][0-9]{{3}})$'
    }
    for format_name, pattern in patterns.items():
        if re.match(pattern, date_str):
            possible_formats.append(format_name)
    
    return possible_formats


df_data = pd.read_excel(r'C:\Users\tomda\OneDrive\Desktop\Python Supper App\Python-ETL\TestDate.xlsx')
df_data.head()
Top100Date = df_data.groupby('Order Date').head(100).reset_index(drop=True)['Order Date']
for i in Top100Date:
    find_possible_date_formats(i)
correct_date_format = max(possible_formats,key=possible_formats.count)
print(correct_date_format)

MM/DD/YYYY


In [3]:

def transform_column_name(col):
    import re
    cleaned_col = re.sub(r'\s+', ' ',re.sub(r'[^a-zA-Z\s]', '', col)).strip().lower()
    cleaned_col = cleaned_col.replace(' ', '_')
    return cleaned_col
cols_sql = []
default_amount = ['amount', 'sales', 'quantity', 'discount', 'profit']
default_date = ['date']
j = 1
for i in df_data.columns:
    if transform_column_name(i) not in l:
        cols_sql.append(transform_column_name(i))
    else:
        cols_sql.append(transform_column_name(i) + '_' + str(j))
        j =+ 1
Amt_col = []
Date_col = []
for i in df_data.columns:
    for j in default_amount:
        if j in transform_column_name(i):
            Amt_col.append(i)
    for j in default_date:
        if j in transform_column_name(i):
            Date_col.append(i)
df_data['Order Date'] = df_data['Order Date'].astype(str)
try:

    df_data[['DayTrans', 'MonthTrans', 'YearTrans']] = df_data['Order Date'].str.split('/', expand=True)
except:
    df_data[['DayTrans', 'MonthTrans', 'YearTrans']] = df_data['Order Date'].str.split('-', expand=True)
if df_data['DayTrans'].astype(int).max() <= 12:
    print('mm/dd/yyyy') #month => mm-dd-yyyy
elif df_data['MonthTrans'].astype(int).max() <= 12 and df_data['YearTrans'].astype(int).max() >= 2000:
    print('dd/mm/yyyy') #month => dd/mm/yyyy 
elif df_data['MonthTrans'].astype(int).max() <= 12 and df_data['DayTrans'].astype(int).max() >= 2000:
    print('yyyy/mm/dd') #month => yyyy-mm-dd


NameError: name 'l' is not defined

In [66]:
class TransformData:
    import pandas as pd
    import numpy as np
    import re
    import openpyxl
    # List month name
    months = r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|January|February|March|April|May|June|July|August|September|October|November|December)'
    possible_formats = []

    def __init__(self, fileLocation):
        self.fileLocation = fileLocation  
    def find_possible_date_formats(date_str):

    # Pattern to determine date format in string
        patterns = {
            'YYYY-MM-DD': r'^([2-9][0-9]{3})-(1[0-2]|0?[1-9])-(3[01]|[12][0-9]|0?[1-9])$',
            'DD-MM-YYYY': r'^(3[01]|[12][0-9]|0?[1-9])-(1[0-2]|0?[1-9])-([2-9][0-9]{3})$',
            'MM-DD-YYYY': r'^(1[0-2]|0?[1-9])-(3[01]|[12][0-9]|0?[1-9])-([2-9][0-9]{3})$',
            'YYYY/MM/DD': r'^([2-9][0-9]{3})/(1[0-2]|0?[1-9])/(3[01]|[12][0-9]|0?[1-9])$',
            'DD/MM/YYYY': r'^(3[01]|[12][0-9]|0?[1-9])/(1[0-2]|0?[1-9])/([2-9][0-9]{3})$',
            'MM/DD/YYYY': r'^(1[0-2]|0?[1-9])/(3[01]|[12][0-9]|0?[1-9])/([2-9][0-9]{3})$',
            'YYYY.MM.DD': r'^([2-9][0-9]{3})\.(1[0-2]|0?[1-9])\.(3[01]|[12][0-9]|0?[1-9])$',
            'DD.MM.YYYY': r'^(3[01]|[12][0-9]|0?[1-9])\.(1[0-2]|0?[1-9])\.([2-9][0-9]{3})$',
            'MM.DD.YYYY': r'^(1[0-2]|0?[1-9])\.(3[01]|[12][0-9]|0?[1-9])\.([2-9][0-9]{3})$',
            'DD Month YYYY': rf'^(3[01]|[12][0-9]|0?[1-9]) {months} ([2-9][0-9]{{3}})$'
        }
        for format_name, pattern in patterns.items():
            if re.match(pattern, date_str):
                possible_formats.append(format_name)
        return possible_formats


b = TransformData()